# Imports

In [1]:
import random

# Question 1 - Next Generation Sequencing Simulator

Develop a program to simulate next-generation sequencing (NGS). Your program will generate simulated DNA sequence reads. It should take an input DNA sequence (genome sequence), create multiple copies of it to ensure high coverage, fragment these copies into fixed-length segments (e.g., 500 base pairs), and simulate the sequencing process by generating reads from 5’ end of fragments up to read length (e.g., 100 bp). Your sequencing read should simulate sequencing by synthesis, meaning that your read should be reverse complement to the 5’ end of the fragment. During sequencing you will also introduce sequencing errors. In addition, some of the fragments will not be sequenced. The output of your program should be a FASTA file containing these simulated reads.


Input parameters:

* Input_sequence: The name of the file that has the genome to be sequenced.
* Coverage: Number of copies of the genome sequence to create before fragmentation.
* Insert_size: Length of each fragment (e.g., 500 bp).
* Read_length: The length of each sequencing read (e.g., 100 bp).
* Seq_error: Sequencing error rate (a decimal value between 0 and 1, e.g., 0.02 for 2% error rate).
* Missing_fragment_rate: Probability of not sequencing a fragment (a decimal value between 0 and 1, e.g., 0.1 for 10% missed fragments).

Output:

* A FASTA file containing the simulated DNA sequence reads.

Ex: Input sequence is AAATGGCAAATGCGGCAATTAG (in reality, it will be much longer)

You will create multiple copies of it (based on coverage parameter).

```
AAATGGCAAATGCGGCAATTAG
AAATGGCAAATGCGGCAATTAG
AAATGGCAAATGCGGCAATTAG
AAATGGCAAATGCGGCAATTAG
```

Then generate DNA fragments by chopping these sequencing at random positions with a specific length. In this example, the fragment length is 4 (please note that the first and last fragment in each line might violate fragment length rule.

```
AAAT GGCA AATG CGGC AATT AG
AAA TGGC AAAT GCGG CAAT TAG
AA ATGG CAAA TGCG GCAA TTAG
A AATG GCAA ATGC GGCA ATTA G
```

Then each fragment will be sequenced from their 5’ end. Let’s assume a read length of 3.

AAAT fragment will generate TTT as a sequencing read (i.e., reverse complement of AAA).

ATGC fragment will generate CAT as a sequencing read (i.e., reverse complement of ATG).

You will also skip some of the fragments (based on missing_fragment_rate parameter) so that no reads will be produced from them.

In addition, with a probability parameterized by seq_error, there will be a sequencing error. For instance, the fragment AATG will be sequenced as GTT instead of ATT.

The output will be all sequencing reads in FASTA format as following:

```
>read 1
TTT
>read 2
CAT
>read 3
GTT
…
```

The reads could appear in the file in any order you wish.

Make sure your code is well-documented. All the specified settings about error rate, read length etc. needs to be parameterized. Your code should read from an input file and should store the results into an output file.

For Question 1, please use the Blcap.txt and sample.txt as your input sequence. In order to replicate your output, please provide the following materials:

1. Output file

2. A readme file that contains the parameter (i.e. coverage, insert_size,.., Missing_fragment_rate) to generate the same output, and any special instructions that are required to run your code.

In [3]:
def get_fragments(seq, insert_size, split):
    fragments = []

    while len(seq) > 0:

        if split < insert_size:
            fragments.append(seq[:split])
            split = insert_size
            seq = seq[split:]

        elif insert_size >= len(seq):
            fragments.append(seq)
            seq = ""

        else:
            fragments.append(seq[:insert_size])
            seq = seq[insert_size:]

    return fragments

def reverse_compliment(seq):
    out = ""
    converter = {"A": "T",
                 "T": "A",
                 "C": "G",
                 "G": "C"
                 }
    for c in seq.strip():
        out = converter[c] + out
    return out

def ngs(input_sequence, coverage, insert_size, read_length, seq_error, missing_fragment_rate):
    bases = ["A", "C", "G", "T"]
    fasta = []

    # Generate copies of the input
    copies = []
    for i in range(coverage):
        copies.append(input_sequence)

    # Fragment each copy
    for i, copy in enumerate(copies):
        new = []
        split = random.randint(0, insert_size)
        frags = [frag for frag in get_fragments(copy, insert_size, split) if len(frag) > 0]

        # Drop some fragments
        for j, frag in enumerate(frags):
            check = random.uniform(0, 1)
            if check > missing_fragment_rate and len(frag) >= read_length:

                # Get reads
                read = frag[:read_length]
                read = reverse_compliment(read) # Reverse compliment

                # Sequencing error
                final = ""
                for k, c in enumerate(read):
                    check = random.uniform(0, 1)
                    if check > seq_error:
                        final += c
                    else:
                        rand = random.choice(bases)
                        while rand == c:
                            rand = random.choice(bases)
                        final += rand

                # FASTA format
                fasta.append("read %d>\n%s"%(len(fasta) + 1, final))
    fasta = "\n".join(fasta)
    return fasta

def ngs_from_file(file_in_name):
    name = ".".join(file_in_name.split(".")[:-1])

    # Get parameters
    with open("parameters-%s.txt"%(name), "r") as file:
        content = file.readlines()
        coverage = int(content[0].split(" ")[-1])
        insert_size = int(content[1].split(" ")[-1])
        read_length = int(content[2].split(" ")[-1])
        seq_error = float(content[3].split(" ")[-1])
        missing_fragment_rate = float(content[4].split(" ")[-1])

        # Open sequence file
        with open(file_in_name, "r") as file_in:
            seq = "".join([line.strip() for line in file_in.readlines()])

            # Get reads
            fasta = ngs(seq, coverage, insert_size, read_length, seq_error, missing_fragment_rate)

            # Write to file
            with open("%s-out.fasta"%(name), "w") as file_out:
                file_out.write(fasta)

# Run for both files
for file in ["sample.txt", "Blcap.txt"]:
    ngs_from_file(file)

# Question 2 - Implement the following problems on Rosalind.

## A) Reconstruct a String from its Genome Path (BA3B)

In [4]:
def get_string(sequences):
    out = sequences[0]
    for seq in sequences[1:]:
        if seq[:-1] == out[len(out) - len(seq) + 1:]:
            out += seq[-1]
        else:
            print("ERROR: Mismatch found.")
            return None

    return out

with open("rosalind_ba3b.txt", "r") as file:
    content = [line.strip() for line in file.readlines() if len(line.strip()) > 0]
    print(get_string(content))

GTCCTTCTTTCGCGCAATAGTCTTGACTAGTGTATGCAAAGTGAATCTAAAGGTCTGTTCTTACGAAGAACATCTGCCCTCAACTTTTATTACCGGTGCGCAAGGACATTCTAACTTCCCCCCGCATTCAGGCCTTCCTGTGGCTCCCCTAATGACGAGCACGCTGATCAGCCGGGGTTCAACTGCATAAGCCTGACTGCAGACAGTGAGGCTTAACTTTGAGGGATGGTAAACTTTGAGTGGACGAGTTGGCAAGATCCAGGAGCTATTTAACCGCCCGCGATCTTAGATCTTTTTTCCAAAGCATGTTCGCCTAGCGCCGGAGTTGTCCAGCTTAGGCTTCCGGCGAATGCACAGGGCGTGTTGTGGCATACAACGTACGACCTTACAGCCACATTACTCACCCACTAGACTTACTGTACAACCATCTGCCACTCTTGTTCAAGGGCAGCTAGTCGGCTCCGGTGCAAGGCTGGGCACCACATCCGTACAACACCCCCGGACGATTCCGGGGATTTCCGGATCTCGAACCGCTAAGATCGGTGATAGCTAGTATGTCACATGGGCCGCTGACGATTCTCATGATTCAACTCCGTGCCGCCGCTATCATCTCTGCAAAGTCCATTGTTTGGCATGACGATAGCACATAATTCTCGCAAAGCGTTCCACATATGCAGATTCGCCGTAGGCTCGATTCACAACAAGATCGAGGCCACCCGAGTGTATGGCCATGGCGTAAAACGAAGAGTCGGAATGGGGAAAATCGTCACTGTAGCCTACCAACGTTGCCCTATGGTACACATGGCCCTAGGTCTCTCCTTTAAAAATACAGGGAATCCAAAGATGGCATACGAATCGGACGGATCGTGTTGGAGGTTCGCCTTGCCCCTACTAATCATCACTCGAGCCTAATAGGGCTCCTTTGGACACGCATGGGGTGGGAATGTGCGTGTTGAAGTAAAAGACACAGCTCTGTCGAGCAAGCCAAACTTTACCCAAC

## B) Construct the Overlap Graph of a Collection of k-mers (BA3C)

In [4]:
def overlap(patterns):

    # Set up adjacency list
    adj_list = {}
    for pat in sorted(list(set(patterns))):
        adj_list.update({pat: []})

    # Get adjacent patterns
    for pat_1 in adj_list:
        for pat_2 in patterns:
            if pat_1[1:] == pat_2[:-1]:
                adj_list[pat_1].append(pat_2)

    # Remove empties
    pats = list(adj_list.keys())
    for p in pats:
        if len(adj_list[p]) < 1:
            del adj_list[p]

    return adj_list

with open("rosalind_ba3c.txt", "r") as file:
    content = file.readlines()
    input = [line.strip() for line in content if len(line.strip()) > 0]

    adj_list = overlap(input)
    for pat in adj_list:
        line = "%s -> %s"%(pat, ", ".join(adj_list[pat]))
        print(line)

AAAAAAATGGGCGCGATGCA -> AAAAAATGGGCGCGATGCAT
AAAAAATGGGCGCGATGCAT -> AAAAATGGGCGCGATGCATT
AAAAAGAGTGCACTGCTCGT -> AAAAGAGTGCACTGCTCGTC
AAAAATGGGCGCGATGCATT -> AAAATGGGCGCGATGCATTA
AAAACTGTAAGTTCAGTCCG -> AAACTGTAAGTTCAGTCCGC
AAAAGAGTGCACTGCTCGTC -> AAAGAGTGCACTGCTCGTCA
AAAAGCACAGTCTGTGCGCT -> AAAGCACAGTCTGTGCGCTC
AAAATACCCGAAATGTGGTC -> AAATACCCGAAATGTGGTCT
AAAATGGGCGCGATGCATTA -> AAATGGGCGCGATGCATTAG
AAACATTGTCGGCCTGGAAA -> AACATTGTCGGCCTGGAAAT
AAACGAACGCCACGCGACCT -> AACGAACGCCACGCGACCTT
AAACTAGTTGTAACAAGAAT -> AACTAGTTGTAACAAGAATA
AAACTGTAAGTTCAGTCCGC -> AACTGTAAGTTCAGTCCGCC
AAAGAGTGCACTGCTCGTCA -> AAGAGTGCACTGCTCGTCAG
AAAGCACAGTCTGTGCGCTC -> AAGCACAGTCTGTGCGCTCA
AAAGGAGCCTGGATTCCCTA -> AAGGAGCCTGGATTCCCTAG
AAAGTCGGCGTCATAGACTC -> AAGTCGGCGTCATAGACTCG
AAAGTCTGTCGGGTTTAGAT -> AAGTCTGTCGGGTTTAGATC
AAAGTTGTGGAGATCAAAAA -> AAGTTGTGGAGATCAAAAAG
AAATACCCGAAATGTGGTCT -> AATACCCGAAATGTGGTCTG
AAATACTCCAAAAAAATGGG -> AATACTCCAAAAAAATGGGC
AAATGGGCGCGATGCATTAG -> AATGGGCGCGATGCATTAGG
AAATGTAAGC